In [73]:
# Import Packages
import pyodbc
import pandas as pd
import glob, os, json
import numpy as np 
from pandarallel import pandarallel
import duckdb
pandarallel.initialize(verbose=0)

In [74]:
# Specify JSON Directory
json_dir = '../Batches/[Batch #]'

# Specify the files in the directory
json_pattern = os.path.join(json_dir, '*.json')
file_list = glob.glob(json_pattern)

# Create an empty array for the new DataFrame and Iterate through each file in the Directory to create the new DataFrame
dfs = []
for file in file_list:
    with open(file) as f:
        json_data = pd.json_normalize(json.loads(f.read()), record_path='playlists')
    dfs.append(json_data)
df = pd.concat(dfs)

In [75]:
df['pid'] = df['pid'].astype(str) #Convert pid field to string
df['tracks'] = df['tracks'].astype(str) #Convert tracks field to string
df['tracks'] = df['tracks']+df['pid'] #Concatenate tracks and pid fields

In [76]:
df2 = df.drop(columns=['name', 'collaborative', 'modified_at', 'num_tracks', 'num_albums', 'num_followers', 'num_edits', 'duration_ms', 'num_artists', 'description']) #Drop surplus columns
i = df.columns.get_loc('tracks') #Assign the tracks field to the variable
df3 = df2['tracks'].str.split("}", expand=True) #Split the tracks field into individual tracks in a playlist
df4 = pd.concat([df2.iloc[:, :i], df3, df2.iloc[:, i+1:]], axis=1) #Join the tracks to their assigned playlists

In [77]:
df3 = df4.melt(id_vars= ['pid'], value_vars=df4.columns[2:], var_name="track_num", value_name="track") #Pivot the separate tracks into row values and assign them their track number based on the "pos" field

In [78]:
new = df3['track'].str.split(", '", n=10, expand=True) #Split each track into its individual attributes
df3['posVal'] = new[0]
df3['field2'] = new[1]
df3['field3'] = new[2]
df3['field4'] = new[3]
df3['field5'] = new[4]
df3['field6'] = new[5]
df3['field7'] = new[6]
df3['field8'] = new[7]
df3['description'] = new[8]
# Separate the attribute names from the attribute values
new = df3['field2'].str.split("':", n=2, expand=True)
df3['delete'] = new[0]
df3['artist_name'] = new[1]
new = df3['field3'].str.split("':", n=2, expand=True)
df3['delete2'] = new[0]
df3['track_uri'] = new[1]
new = df3['field4'].str.split("':", n=2, expand=True)
df3['delete3'] = new[0]
df3['artist_uri'] = new[1]
new = df3['field5'].str.split("':", n=2, expand=True)
df3['delete4'] = new[0]
df3['track_name'] = new[1]
new = df3['field6'].str.split("':", n=2, expand=True)
df3['delete5'] = new[0]
df3['album_uri'] = new[1]
new = df3['field7'].str.split("':", n=2, expand=True)
df3['delete6'] = new[0]
df3['track_duration_ms'] = new[1]
new = df3['field8'].str.split("':", n=2, expand=True)
df3['delete7'] = new[0]
df3['album_name'] = new[1]
df3 = df3.dropna(subset='track_name') #Drop all rows that have all null values

In [79]:
df5 = df3.drop(columns=['posVal', 'field2', 'field3', 'field4', 'field5', 'field6', 'delete', 'delete2', 'delete3', 'delete4', 'delete5', 'delete6', 'delete7', 'field7', 'field8','track', 'description']) #Drop all surplus columns
df6 = df5[['pid', 'track_num', 'track_name', 'artist_name', 'album_name', 'track_duration_ms', 'track_uri', 'artist_uri', 'album_uri']] #Reorder columns
#Remove uri headers
df6['track_uri'] = df6['track_uri'].str[-23:-1]
df6['artist_uri'] = df6['artist_uri'].str[-23:-1]
df6['album_uri'] = df6['album_uri'].str[-23:-1]
#Remove unncessary characters
df6['track_name'] = df6['track_name'].str.replace("'", "")
df6['artist_name'] = df6['artist_name'].str.replace("'", "")
df6['album_name'] = df6['album_name'].str.replace("'", "")
df6['track_name'] = df6['track_name'].str.replace('"', "")
df6['artist_name'] = df6['artist_name'].str.replace('"', "")
df6['album_name'] = df6['album_name'].str.replace('"', "")
df6['track_num'] = df6['track_num'].astype('int16') #Convert track_num field to integer
df6['pid'] = df6['pid'].astype('int32') #Convert pid field to integer
final = df6
final #Preview data

,pid,track_num,track_name,artist_name,album_name,track_duration_ms,track_uri,artist_uri,album_uri
0,995000,0,Back To Sleep,Chris Brown,Royalty (Deluxe Version),201253,6dlpABcXrQKRU9G00i6Zba,7bXgB6jMjp9ATFy66eO08Z,0VWmEVuQ8tA5iA3cCTrgxa
1,995001,0,Nights,Frank Ocean,Blonde,307151,7eqoqGkKwgOaWNNHx90uEZ,2h93pZq0e7k5yf4dywlkpM,3mH6qwIy9crq0I9YQbOuDf
2,995002,0,Fantastic Boy,BIGBANG,BIGBANG Special Edition Still Alive 1,232586,2GrYIUrAYK5TZhSh0RJV6H,4Kxlr1PRlDKEB0ekOCyHgX,7tCxOPrGwO3ojcfi8ZdNKQ
3,995003,0,Swang,Rae Sremmurd,SremmLife 2,208120,6mapJIPnQ23RTAevUoE0DL,7iZtZyCzp3LItcw1wtPI3D,5v6iBhIlflzR9rEd1LAMbd
4,995004,0,Machu Picchu,The Strokes,Angles,209626,6mVD1SfTvlFAPVi7txFL5H,0epOFNiUfyON9EYx7Tpr6V,6Jx4cGhWHewTcfKDJKguBQ
...,...,...,...,...,...,...,...,...,...
1237158,997158,247,Elementary,Lil Pump,Elementary,92577,0gdGdUIIXUH3WSLGeXXWw0,3wyVrVrFCkukjdVIdirGVY,7AgxNxn7AAKoxVwj5LQRfM
1237980,997980,247,Closer,The Chainsmokers,Closer,244960,7BKLCZ1jbUBVqRi2FVlTVw,69GGBxA162lTqCwzJG5jLp,0rSLgV8p5FzfnqlEk4GzxE
1239277,999277,247,Conqueror (feat. Estelle and Jussie Smollett),Empire Cast,Original Soundtrack from Season 1 of Empire,268426,7uQztXdiRLRu4YSviPqfD9,7EjxNoVjXo7gL73F2EdXVA,65nrMZ7EnJgYKeasDhRXTy
1239592,999592,247,Push It,Salt-N-Pepa,"Hot, Cool & Vicious",272133,0TT7wJiEYD5GAeJfSR1ETX,7wqtxqI3eo7Gn1P7SpP6cQ,1zSDLZLSN9nEJwmkrahZkl


In [80]:
#Establish connection to SQL Server
server = 'LAPTOP-E80R5A71' 
database = 'Spotify' 
username = 'test' 
password = 'Dondiablo112!' 
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()
# Insert Dataframe into SQL Server:
for index, row in final.iterrows():
     cursor.execute("INSERT INTO master (pid, track_num, track_name, artist_name, album_name, track_duration_ms, track_uri, artist_uri, album_uri) values(?,?,?,?,?,?,?,?,?)", row.pid, row.track_num, row.track_name, row.artist_name, row.album_name, row.track_duration_ms, row.track_uri, row.artist_uri, row.album_uri)
cnxn.commit()
cursor.close()